In [1]:
import pandas as pd
import numpy as np
from meteostat import Point, Stations, Hourly
import datetime

from great_tables import GT, md, style, loc

import altair as alt
from vega_datasets import data

# using a linear model to predict the weather at Pittsburgh international airport  

the features used to predict the weather are the wind speed and direction for the following weather stations 

- Allegheny County Airport
- Butler / Brownsdale
- Washington / Lagonda
- Zelienople / Old Furnace
-  Beaver Falls / West Mayfield

the values we are trying to predict is the wind speed and direction for Greater Pittsburgh International Airport

the model got the folowing metrics on the test set.      

- R²: 0.620057846963885
- RMSE: 54.6469878681952

see the last code cell to see a visualization of the predicted vs actual wind speed.

potential improvements:

- convert wind vector from angle and speed to its north-south and east-west components.
- change more complex model.
- predict more locations (given there closest stations)
- add more features 






In [2]:
stations = Stations()
stations = stations.nearby(40.4406, -79.9959)

stations = stations.fetch(6)

data = Hourly(stations, datetime.datetime(2024, 1, 1), datetime.datetime(2024, 12, 31))
data = data.fetch()
data = data.reset_index()[["station","time", "wspd","wdir"]]


data = data.pivot(index="time", columns="station", values=["wspd", "wdir"])

# flatten MultiIndex columns: wspd_XXXX, wdir_XXXX
data.columns = [f"{var}_{station}" for var, station in data.columns]
data = data.dropna()


X = data[['wspd_F2UX6', 'wspd_KAFJ0', 'wspd_KBTP0', 'wspd_KBVI0',
        'wspd_KPJC0',  'wdir_F2UX6', 'wdir_KAFJ0', 'wdir_KBTP0',
        'wdir_KBVI0', 'wdir_KPJC0']]

y = data[["wspd_72520", "wdir_72520"]]

stations = stations.reset_index()
print(stations["name"])


0    Greater Pittsburgh International
1            Allegheny County Airport
2                 Butler / Brownsdale
3                Washington / Lagonda
4            Zelienople / Old Furnace
5        Beaver Falls / West Mayfield
Name: name, dtype: object


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# --------------------------
#  X and y are already built
# --------------------------

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,      # 20% test
    shuffle=True,
    random_state=42
)

# --------------------------
# Fit linear regression model
# --------------------------
model = LinearRegression()
model.fit(X_train, y_train)

# --------------------------
# Predict
# --------------------------
y_pred = model.predict(X_test)

# --------------------------
# Metrics
# --------------------------
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

#
coef_table = dict(zip(X.columns, model.coef_[0]))
print("Coefficients:", coef_table)
y_test = y_test.to_numpy()


R²: 0.620057846963885
RMSE: 54.6469878681952
Coefficients: {'wspd_F2UX6': np.float64(0.13055504149719682), 'wspd_KAFJ0': np.float64(0.2774017968854069), 'wspd_KBTP0': np.float64(0.37181429144890094), 'wspd_KBVI0': np.float64(0.0988597008315906), 'wspd_KPJC0': np.float64(0.1347943680084977), 'wdir_F2UX6': np.float64(0.009695508700787836), 'wdir_KAFJ0': np.float64(0.0009781991390266642), 'wdir_KBTP0': np.float64(0.00524996999557395), 'wdir_KBVI0': np.float64(-0.001846885448983343), 'wdir_KPJC0': np.float64(0.001854613223023563)}


In [4]:

# Convert to NumPy array if not already
y_test = np.array(y_test, dtype=float)  # ensures it's numeric
y_pred = np.array(y_pred, dtype=float)

# Now compute observed components
u_obs = y_test[:, 0] * np.sin(np.deg2rad(y_test[:, 1]+180))
v_obs = y_test[:, 0] * np.cos(np.deg2rad(y_test[:, 1]+180))

# Predicted components
u_pred = y_pred[:, 0] * np.sin(np.deg2rad(y_pred[:, 1]+180))
v_pred = y_pred[:, 0] * np.cos(np.deg2rad(y_pred[:, 1]+180))


wind_df = pd.DataFrame({
    'u_obs': u_obs,
    'v_obs': v_obs,
    'u_pred': u_pred,
    'v_pred': v_pred
})
# Get scalar longitude/latitude
lon = stations.loc[stations["id"]=="72520", "longitude"].values[0]
lat = stations.loc[stations["id"]=="72520", "latitude"].values[0]

# Repeat for all rows in wind_df
wind_df["longitude"] = lon
wind_df["latitude"] = lat
wind_df

,u_obs,v_obs,u_pred,v_pred,longitude,latitude
0,2.599353e+00,7.141664e+00,1.413913,8.209047,-80.0667,40.4833
1,6.442347e+00,-3.653637e+01,12.164918,-22.120161,-80.0667,40.4833
2,8.140639e+00,4.700000e+00,10.199447,10.454138,-80.0667,40.4833
3,1.549118e+01,-1.846167e+01,16.486029,-10.785461,-80.0667,40.4833
4,2.230000e+01,6.827406e-15,19.445381,8.042900,-80.0667,40.4833
...,...,...,...,...,...,...
1704,-1.221600e+01,4.446262e+00,-8.935751,6.088810,-80.0667,40.4833
1705,4.114813e-15,-1.120000e+01,11.113174,-6.406085,-80.0667,40.4833
1706,1.775352e+01,-1.025000e+01,13.922873,-3.256108,-80.0667,40.4833
1707,9.200000e+00,-1.593487e+01,9.780221,-5.859170,-80.0667,40.4833


In [ ]:
# calculate vectors  
scale = 0.025  
# Convert to NumPy array if not already
y_test = np.array(y_test, dtype=float)  # ensures it's numeric
y_pred = np.array(y_pred, dtype=float)

# Now compute observed components
u_obs = y_test[:, 0] * np.sin(np.deg2rad(y_test[:, 1]+180))
v_obs = y_test[:, 0] * np.cos(np.deg2rad(y_test[:, 1]+180))

# Predicted components
u_pred = y_pred[:, 0] * np.sin(np.deg2rad(y_pred[:, 1]+180))
v_pred = y_pred[:, 0] * np.cos(np.deg2rad(y_pred[:, 1]+180))


wind_df = pd.DataFrame({
    'u_obs': u_obs,
    'v_obs': v_obs,
    'u_pred': u_pred,
    'v_pred': v_pred
})
# Get scalar longitude/latitude
lon = stations.loc[stations["id"]=="72520", "longitude"].values[0]
lat = stations.loc[stations["id"]=="72520", "latitude"].values[0]

# Repeat for all rows in wind_df
wind_df["longitude"] = lon
wind_df["latitude"] = lat
wind_df["end_lon"] = wind_df["longitude"] + wind_df["u_obs"] * scale
wind_df["end_lat"] = wind_df["latitude"] + wind_df["v_obs"] * scale

wind_df["end_lon_pred"] = wind_df["longitude"] + wind_df["u_pred"] * scale
wind_df["end_lat_pred"] = wind_df["latitude"] + wind_df["v_pred"] * scale




In [ ]:


wind_df = wind_df.reset_index().rename(columns={"index": "arrow_id"})

In [8]:

# --- slider parameter ---
slider = alt.binding_range(
    min=0,
    max=len(wind_df)-1,
    step=1,
    name="Arrow:"
)

selector = alt.param(
    name="selected_arrow",
    bind=slider,
    value=0
)

# --- map base ---
us_states = alt.topo_feature(data.us_10m.url, 'states')
PA_FIPS = "42"

base = (
    alt.Chart(us_states)
    .mark_geoshape(fill="#f0f0f0", stroke="black")
    .transform_filter(f"datum.id == '{PA_FIPS}'")
    .project("albersUsa")
)

# --- stations ---
points = (
    alt.Chart(stations)
    .mark_circle(size=200, opacity=0.8)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        color="name:N",
        tooltip=["name:N", "id:N"]
    )
)

# --- wind arrow (filtered to one row) ---
arrows = (
    alt.Chart(wind_df)
    .mark_line()
    .encode(
        shape=alt.ShapeValue('arrow'),
        longitude='longitude:Q',
        latitude='latitude:Q',
        longitude2='end_lon:Q',
        latitude2='end_lat:Q',
        color= alt.value("blue"),
         strokeWidth=alt.value(4)
    )
    .add_params(selector)
    .transform_filter("datum.arrow_id == selected_arrow")
)

arrows_pred =(
    alt.Chart(wind_df)
    .mark_line()
    .encode(
        shape=alt.ShapeValue('arrow'),
        longitude='longitude:Q',
        latitude='latitude:Q',
        longitude2='end_lon_pred:Q',
        latitude2='end_lat_pred:Q',
        color= alt.value("red"),
        strokeWidth=alt.value(4)
    )
    .add_params()
    .transform_filter("datum.arrow_id == selected_arrow")
)

# --- final plot ---
(base + points + arrows + arrows_pred).properties(
    width=600,
    height=400,
    title="Observed vs predicted Wind Vector"
)


AttributeError: 'DataFrame' object has no attribute 'us_10m'